# Identification of Lindblad form Markovian ME
## (for spin-boson system using TSSOS POP method)  

In [1]:
include("LiPoSID.jl")

using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra

using HDF5
using DynamicPolynomials

using Dates

using Statistics

using TSSOS

In this notebook we identify and test accuracy of Markovian master equations in Lindblad form for the nontrivial example of the data generated by the full-featured simulation of the spin-boson model (two-level system coupled with the bath of the continuum of the harmonic oscillators).  


Consider simple Lindblad master equation with two dissipators:

$
    \frac{d\rho}{dt} = \mathcal{L}[\rho(t)] = -\frac{i}{\hbar} [H,\rho(t)] + \mathcal{D}[\rho(t)] = - \frac{i}{\hbar}[H, \rho]+\sum_{\ell=1}^{s-1}\left[J_\ell \rho J_\ell^\dagger - \frac{1}{2}\left\{ J_\ell^\dagger J_\ell, \rho \right\} \right],
$

In [2]:
function lindblad_rhs(ρ, H, J::Array)
       """
       Right hand side of the Lindblad master equation with multiple disipators
       """
      
       Σ = sum([ ( Jⱼ * ρ * Jⱼ' - (Jⱼ' * Jⱼ  * ρ + ρ * Jⱼ' * Jⱼ)/2 ) for Jⱼ in J ])
       
       return -im * (H * ρ - ρ * H) + Σ 
       
   end

lindblad_rhs (generic function with 1 method)

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} \epsilon & \alpha + i \beta \\ \alpha - i \beta  & -\epsilon
   \end{pmatrix}/2
$

here we consider the following set of jump operators ot the particular ansatz form:

$
J_1 = \begin{pmatrix} 0 & r \\ 0 & 0 
   \end{pmatrix} = 
   \begin{pmatrix} 0 & \sqrt{\gamma_1} \\ 0 & 0 
   \end{pmatrix}
$  - for relaxation process, 

and three following operators for depolarization process:

$
J_2 = \begin{pmatrix} 0 & p \\ p & 0 
   \end{pmatrix} = \sqrt{\gamma_4/3} 
   \begin{pmatrix} 0 & 1\\ 1 & 0 
   \end{pmatrix}
$

$
J_3 = \begin{pmatrix} 0 & -i p \\ i p & 0 
   \end{pmatrix} = \sqrt{\gamma_4/3}
   \begin{pmatrix} 0 & -i \\ i & 0 
   \end{pmatrix}
$

$
J_4 = \begin{pmatrix} p & 0 \\ 0 & -p
   \end{pmatrix} = \sqrt{\gamma_4/3}
   \begin{pmatrix} 1 & 0 \\ 0 & -1 
   \end{pmatrix}
$

In [3]:
# Define polynomial variables
@polyvar ϵ α β r p

Hˢʸᵐᵇ = [ ϵ            α + im* β
          α - im* β   -ϵ         ]/2

J₁ˢʸᵐᵇ = [ 0   r
           0   0. ]

J₂ˢʸᵐᵇ = p * [ 0    1
               1    0. ]

J₃ˢʸᵐᵇ = p * [ 0   -im
               im   0  ]

J₄ˢʸᵐᵇ = p * [ 1    0
               0   -1. ]


Jˢʸᵐᵇ = [ J₁ˢʸᵐᵇ, J₂ˢʸᵐᵇ, J₃ˢʸᵐᵇ, J₄ˢʸᵐᵇ]

4-element Vector{Matrix}:
 Term{true, Float64}[0.0 r; 0.0 0.0]
 Term{true, Float64}[0.0 p; p 0.0]
 Term{true, Complex{Int64}}[(0 + 0im) (0 - 1im)p; (0 + 1im)p (0 + 0im)]
 Term{true, Float64}[p 0.0; 0.0 -p]

To ensure the objective is polynomial we can write loss function with double step and using the Frobenius norm :

$ L = \sum_{j=3}^N
{\left\| \rho_j - \rho_{j-2} - {\mathcal{L}} \left[\int^{t_j}_{t_{j-2}}\rho(t)dt \right] \right\| }^2_F $

and apply the Simpson approximation method to evaluate the integral:

$
\int^{t_j}_{t_{j-2}}\rho(t)dt = \frac{1}{3} \Delta t \left[ \rho_{j-2} + 4 \rho_{j-1} + \rho_j \right] +  \mathcal{O}(\Delta t^5).$

Using it we can calculate objective as polynomial of nine variables  and cast the optimization problem:

$
\min L\left( \epsilon, \alpha, \beta, r, p, \{\rho_k\} \right)
$

In [4]:
function simpson_obj(ρ, t, H, J)
    
    obj = 0
    for i in 3:length(ρ)
        obj += LiPoSID.frobenius_norm2(
            ρ[i] - ρ[i-2] - (t[i]-t[i-1])lindblad_rhs((ρ[i-2] + 4ρ[i-1] + ρ[i])/3, H, J)
        )
    end
    obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    return obj
end

simpson_obj (generic function with 1 method)

Density matrices for four bases initial states $\ket{0}$, $\ket{1}$, $\ket{x}$, $\ket{y}$ that were used to simulate training data are defined as follows:
\begin{align}
   \rho^{(g)}_0 &=  \rho^{\ket{0}}_0 = \left(
\begin{array}{c}
            1  \\
            0   \\
        \end{array}
                \right) 
                \left(
                \begin{array} {cc}
                1 & 0
                \end{array}
                \right) = 
\ket{0} \bra{0} = \left(
        \begin{array}{ccccc}
            1  &  0  \\
            0  &  0   \\
        \end{array}
            \right),\\
\rho^{(e)}_0 &=\rho^{\ket{1}}_0 =\ket{1} \bra{1} = 
\begin{pmatrix}
0 \\ 1
\end{pmatrix}
\begin{pmatrix}
0 & 1
\end{pmatrix}=
\left(
        \begin{array}{ccccc}
            0  &  0  \\
            0  &  1  \\
        \end{array}
            \right), \notag\\
\rho^{\ket{x}}_0 &=\ket{x} \bra{x} = |+\rangle \langle + | = \left(
\begin{array}{c}
            1/\sqrt{2}  \\
            1/\sqrt{2}   \\
        \end{array}
                \right) 
                \left(
                \begin{array} {cc}
                1/\sqrt{2} & 1/\sqrt{2} 
                \end{array}
                \right) = \frac{1}{2}\left(
        \begin{array}{ccccc}
            1  &  1  \\
            1  &  1   \\
        \end{array}
    \right), \notag\\
\rho^{\ket{y}}_0  &= \ket{y} \bra{y} = |-\rangle \langle-| = \left(
\begin{array}{c}
            1/\sqrt{2}  \\
            i/\sqrt{2}   \\
        \end{array}
                \right) 
                \left(
                \begin{array} {cc}
                1/\sqrt{2} & -i/\sqrt{2} 
                \end{array}
                \right) = \frac{1}{2}\left(
        \begin{array}{ccccc}
            1  &  -i  \\
            i  &  1   \\
        \end{array}
    \right)
\end{align}

The function `lindblad_GEXY_obj()` 
below combines the objective for the for data series of density matrices starting in four basis states 
$\ket{0}$, $\ket{1}$, $\ket{x}$, $\ket{y}$:

In [5]:
function lindblad_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Jˢʸᵐᵇ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    polyG = simpson_obj(ρᵍ, tᵍ, Hˢʸᵐᵇ, Jˢʸᵐᵇ)
    polyE = simpson_obj(ρᵉ, tᵉ, Hˢʸᵐᵇ, Jˢʸᵐᵇ)
    polyX = simpson_obj(ρˣ, tˣ, Hˢʸᵐᵇ, Jˢʸᵐᵇ)
    polyY = simpson_obj(ρʸ, tʸ, Hˢʸᵐᵇ, Jˢʸᵐᵇ)

    polyGEXY = polyG + polyE + polyX + polyY

    return polyGEXY
end

lindblad_GEXY_obj (generic function with 1 method)

In [7]:
constraints = [ϵ, p] #r, e, ϕ, p]

2-element Vector{PolyVar{true}}:
 ϵ
 p

In [8]:
# Here we set parameters what data to use for training and testing

evol_data_file_name = "DATA/ALL_GAMMAS_B4_D10.h5"

tests_dir = ""

ρᵍ₀ = [ 1 0
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];
basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states

all_states = vcat(train_states, test_states);

Checking functions on one particular coupling level

In [11]:
γᵢ = "0.25133"

tᵍᵉˣʸ , ρᵍᵉˣʸ  = LiPoSID.read_GEXY_timeevolution(evol_data_file_name, γᵢ)
polyGEXYfull = lindblad_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Jˢʸᵐᵇ)

0.18256809044653088r⁴ - 0.03750921946960905r²p² + 17.65754243030726p⁴ + 0.0012278271223203374αr² - 0.0008250182359115586βr² + 0.10341163322607885ϵ² - 4.104630886766604e-5ϵα + 3.451755329501484e-5ϵβ + 1.0519096548020683α² + 0.000517967623618515αβ + 1.0518715157602614β² - 0.09164160023232439r² + 0.0032558121009390465p² - 5.199679015672933ϵ + 0.00013669789552732257α - 0.00035392288736811714β + 65.37495321955478

In [12]:
variables(polyGEXYfull)'

1×5 adjoint(::Vector{PolyVar{true}}) with eltype PolyVar{true}:
 ϵ  α  β  r  p

In [13]:
LiPoSID.coefficient_range(polyGEXYfull)

5.279935448533528e-7

Below is the wrapper function to call the TSSOS library function for the constraint polynomial optimization:

In [14]:
function min_cs_tssos(p, constrs)

    coeffs = coefficients(p)
    reg_coef = 0

    pop =[p+reg_coef*sum(variables(p).^2), constrs...] ./ maximum(abs.(coeffs))

    d = maxdegree(p)
    
    # Initial optimization step
    opt, sol, data = cs_tssos_first(pop, variables(pop), d; solution=true, QUIET=true)
    ref_sol, flag = TSSOS.refine_sol(opt, sol, data; QUIET=true)
    prev_opt, prev_sol, prev_data = opt, sol, data 

    # Check if the solution needs further refinement
    if flag != 0
        while ~isnothing(sol) && flag != 0
            prev_opt, prev_sol, prev_data = opt, sol, data
            opt, sol, data = cs_tssos_higher!(data; solution=true, QUIET=true) 
        end
        ref_sol, flag = TSSOS.refine_sol(prev_opt, prev_sol, prev_data; QUIET=true)
    end

    solution = variables(p) => ref_sol

    if flag == 0 
        status_name = "GLOBAL"
    else
        status_name = "LOCAL/FAIL"
    end

    return solution, status_name

end

min_cs_tssos (generic function with 1 method)

In [15]:
sol, st = min_cs_tssos(polyGEXYfull, constraints)

*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[3, 2]
[2, 1]
-----------------------------------------------------------------------------
termination status: SLOW_PROGRESS
solution status: NO_SOLUTION
optimum = 6.963919247976713e-9

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Global optimality certified with relative optimality gap 0.002601%!
Global optimality certified with relative optimality gap 0.002601%!


(PolyVar{true}[ϵ, α, β, r, p] => [25.140687157086514, 0.0002790776568615105, -0.00014590158300080663, 0.5009961700124128, 0.013669214257062093], "GLOBAL")

In [16]:
Hˢⁱᵈ = convert.(ComplexF64,subs(Hˢʸᵐᵇ, sol))
Hˢⁱᵈ[1:2,1:2]

2×2 Matrix{ComplexF64}:
     12.5703+0.0im         0.000139539-7.29508e-5im
 0.000139539+7.29508e-5im     -12.5703+0.0im

In [17]:
sol, st = min_cs_tssos(LiPoSID.filter_odd_terms_by_relative_threshold(polyGEXYfull, 1e-4), constraints)

*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[2, 1]
[1, 3]
-----------------------------------------------------------------------------
termination status: SLOW_PROGRESS
solution status: NO_SOLUTION
optimum = 7.455967800350148e-10
Global optimality certified with relative optimality gap 0.002602%!
Global optimality certified with relative optimality gap 0.002602%!


(PolyVar{true}[ϵ, α, β, r, p] => [25.140687077350453, 0.0, 0.0, 0.5009974357225474, 0.013669260403264221], "GLOBAL")

In [18]:
st

"GLOBAL"

In [19]:
J₁ˢⁱᵈ = convert.(ComplexF64,subs(J₁ˢʸᵐᵇ, sol))
J₁ˢⁱᵈ[1:2,1:2]

2×2 Matrix{ComplexF64}:
 0.0+0.0im  0.500997+0.0im
 0.0+0.0im       0.0+0.0im

In [20]:
Hˢⁱᵈ = convert.(ComplexF64, subs(Hˢʸᵐᵇ, sol))

2×2 Matrix{ComplexF64}:
 12.5703+0.0im       0.0+0.0im
     0.0+0.0im  -12.5703+0.0im

Below we perform system identification and testing of the accuracy of identified models for all available coupling levels of spin boson system:

In [22]:
println(" MULTI DURATION SYSTEM IDENTIFICATION w CONSTRAINED TSSOS and LINDBLAD 4ops Frobenius objective QO simulation")

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

date_and_time_string = string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

evol_data_file_name = "DATA/ALL_GAMMAS_B4_D10.h5"

relative_threshold = 1e-15

rltrs = string(convert(Int, floor(log10(relative_threshold))))

tests_data_file_name = "MULT_LINDBLAD_CONSTR_TSSOS_threshold_1e"*rltrs*"_FROB_QO_"*date_and_time_string * ".h5"

FminGammas = []
FmedianGammas = []
FmeanGammas = []
Epsilons = []
CoefRanges = []

for γᵢ in γ
    println("γ =  "*γᵢ)

    # Save results into HDF5
    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, γᵢ) # Create gamma coupling group
    end
    
    # Read time series data for the current γ
    tᵍᵉˣʸ, ρᵍᵉˣʸ = LiPoSID.read_GEXY_timeevolution(evol_data_file_name, γᵢ)
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ
    
    # Find the end of the shortest time series
    t_end = minimum([tᵍ[end], tᵉ[end], tˣ[end], tʸ[end]])

    # Dynamically generate the duration steps
    max_steps = Int(floor(t_end * 2 * parse(Float64, γᵢ)))  # Maximum steps based on t_end and γᵢ
    duration_steps = [i/(2*parse(Float64, γᵢ)) for i in 1:max_steps]  # Generates dynamic steps up to t_end
    
    # Ensure steps do not exceed the shortest time series
    duration_steps = duration_steps[duration_steps .<= t_end]

    for t_duration in duration_steps
        println("Processing time duration: ", t_duration)

        elapsed_time = @timed begin
            # Masking the time series up to t_duration
            t_masked = [t[t .<= t_duration] for t in [tᵍ, tᵉ, tˣ, tʸ]]
            min_length = minimum(length.(t_masked))  # Ensure all time series are truncated to the same length
            t_masked = [t[1:min_length] for t in t_masked]

            ρ_masked = [ρ[1:min_length] for ρ in ρᵍᵉˣʸ] # Truncate ρᵍᵉˣʸ accordingly

            # Objective function construction and minimization
            polyGEXYfull = lindblad_GEXY_obj(ρ_masked, t_masked, Hˢʸᵐᵇ, Jˢʸᵐᵇ)
            polyGEXY = LiPoSID.filter_odd_terms_by_relative_threshold(polyGEXYfull, relative_threshold)
            sol, status = min_cs_tssos(polyGEXY, constraints)
        end

        @show minimum(abs.(coefficients(polyGEXY)))
        @show maximum(abs.(coefficients(polyGEXY)))
        push!(CoefRanges, LiPoSID.coefficient_range(polyGEXYfull))

        print(" status:", status)
        print(" runtime :", elapsed_time.time)

        Hˢⁱᵈ = convert.(ComplexF64, subs(Hˢʸᵐᵇ, sol))
        Jˢⁱᵈ = [convert.(ComplexF64,subs(Jᵢˢʸᵐᵇ, sol)) for Jᵢˢʸᵐᵇ in Jˢʸᵐᵇ]

        epsilon = subs(ϵ, sol)
        push!(Epsilons, epsilon)

        # Save results into HDF5
        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, γᵢ) # Create gamma coupling group
            t_group = create_group(γ_group, string(t_duration)) # Create time duration group
            t_group["epsilon"] = convert(Float64, epsilon)
            t_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)
            t_group["J1"] = convert.(ComplexF64, Jˢⁱᵈ[1])
            t_group["J2"] = convert.(ComplexF64, Jˢⁱᵈ[2])
            t_group["J3"] = convert.(ComplexF64, Jˢⁱᵈ[3])
            t_group["J4"] = convert.(ComplexF64, Jˢⁱᵈ[4])
            t_group["status"] = status
            t_group["runtime"] = elapsed_time.time
        end

        println()

        FminStates = []
        FmedianStates = []
        FmeanStates = []

        print("Processing states: ")

        for state in test_states # Loop over initial states
            print(state*" ")
            start_time = time()

            tₛ, ρₛ = LiPoSID.read_timeevolution(evol_data_file_name, state, γᵢ)
            ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
            ρᵗˢᵗ = [DenseOperator(basis, Hermitian(ρₜ)) for ρₜ in ρₛ]
            tᵗˢᵗ = convert(Vector{Float64}, tₛ)

            ρₒ = DenseOperator(basis, ρₛ[1])
            dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
            tᵉⁿᵈ = tᵗˢᵗ[end]

            effective_Lindblad = Jˢⁱᵈ
            effective_Lindblad_ops = [DenseOperator(basis, j) for j in effective_Lindblad]

            tout, ρ_t_kossak = timeevolution.master(convert.(Float64, tᵗˢᵗ), ρₒ, DenseOperator(basis, Hˢⁱᵈ), effective_Lindblad_ops)
            ρˢⁱᵈ = [ρₜ.data for ρₜ in ρ_t_kossak]

            F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)

            FminState = minimum(F)
            FmedianState = mean(F)
            FmeanState = mean(F)

            h5open(tests_dir*tests_data_file_name,"cw") do fid
                γ_group = open_group(fid, γᵢ)
                t_group = open_group(γ_group, string(t_duration))
                init_state_group = create_group(t_group, state)
                init_state_group["Fidelity"] = convert.(Float64, F)
                init_state_group["Fmin"] = convert(Float64, FminState)
                init_state_group["Fmedian"] = convert(Float64, FmedianState)
                init_state_group["Fmean"] = convert(Float64, FmeanState)

                init_state_group["tr_dist_grnd"] = LiPoSID.TrDist(ρₛ[1], ρᵍ₀)
                init_state_group["time"] = tᵗˢᵗ
            end

            push!(FminStates, FminState)
            push!(FmedianStates, FmedianState)
            push!(FmeanStates, FmeanState)
        end

        F_mean_value = mean(FmeanStates)
        F_median_value = median(FmedianStates)
        F_min_value = minimum(FminStates)

        push!(FminGammas, F_min_value)
        push!(FmedianGammas, F_median_value)
        push!(FmeanGammas, F_mean_value)

        println("Median fidelity for γ = "*γᵢ*" and time duration "*string(t_duration)*": ", F_median_value)

    end
end
    
    
println(tests_data_file_name)

 MULTI DURATION SYSTEM IDENTIFICATION w CONSTRAINED TSSOS and LINDBLAD 4ops Frobenius objective QO simulation
γ =  0.079477
Processing time duration: 6.291128250940523
*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[3, 2]
[2, 1]
-----------------------------------------------------------------------------
termination status: SLOW_PROGRESS
solution status: NO_SOLUTION
optimum = 7.012549719112515e-9
Global optimality certified with relative optimality gap 0.002691%!
Global optimality certified with relative optimality gap 0.000252%!
minimum(abs.(coefficients(polyGEXY))) = 0.00018176880830710583
maximum(abs.(coefficients(polyGEXY))) = 226.05632125567843
 status:GLOBAL runtime :1.022256815
Processing states: D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for γ = 0.079477 and time duration 6.291128250940523: 0.999603474928597
Proces

In [24]:
println(tests_data_file_name)

MULT_LINDBLAD_CONSTR_TSSOS_threshold_1e-15_FROB_QO_2024-Sep-29_at_21-07.h5
